In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [43]:
from llama_index.core.chat_engine import ContextChatEngine
from llama_index_client import ChatMessage, MessageRole
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings
from llama_index.core.memory import ChatMemoryBuffer

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

# create index with Documents + LLM
index2 = VectorStoreIndex.from_documents(
    documents, embed_model=Settings.embed_model, llm=Settings.llm
)

chatRetreiver = index2.as_retriever()
memory = ChatMemoryBuffer.from_defaults(token_limit=5500)

messageHistory =  [ 
                    ChatMessage(
                        role=MessageRole.SYSTEM,
                        content=
                                "You are an expert assitant for extracting insights from resume in PDF format. \n"
                                "You extract data and return it in JSON format, as below \n"
                                "  {\n"
                                "    CompanyName1:\n"
                                "    JobTitle:\n"
                                "    JobPeriod:\n"
                                "  }\n"
                                "REMEMBER to return extracted data only from provided resume.",
                        additional_kwargs={"2":"2"}
                        ),
                    ChatMessage(
                        role=MessageRole.USER,
                        content="Hello assistant, we are having a insightful discussion about Ramya Anand today.",
                        additional_kwargs={"2":"2"}
                        ),
                    ChatMessage(
                        role=MessageRole.ASSISTANT, 
                        content="Okay, sounds good.",
                        additional_kwargs={"2":"2"}
                        ),
                  ]

newMessage = "List the companies Ramya Anand has worked, her role, job period in Json format?"

chatEngine = ContextChatEngine(chatRetreiver, Settings.llm, memory= memory, prefix_messages= messageHistory)
response3 = chatEngine.chat(newMessage)
print(response3)


{
    "CompanyName1": "Modaxo (Constellation Software, Canada)",
    "JobTitle": "Full stack Developer",
    "JobPeriod": "06/2021 - 04/2024"
},
{
    "CompanyName2": "Virtusa (CIBC - Capital Market, Canada)",
    "JobTitle": "Senior Application Developer",
    "JobPeriod": "02/2020 - 06/2021"
}


In [67]:
response4 = chatEngine.chat("What are the other companies she worked for apart from the ones listed already? \n"
                            "Return the response in same Json format, continue with the company serial numbers")
print(response4)

{
    "CompanyName3": "Bank of NewYork Mellon, US / India",
    "JobTitle": "Lead - Application Development",
    "JobPeriod": "07/2017 - 12/2019"
  },
  {
    "CompanyName4": "iNautix Technologies, US / India",
    "JobTitle": "Technical Lead",
    "JobPeriod": "07/2014 - 06/2017"
  },
  {
    "CompanyName5": "iNautix Technologies, US / India",
    "JobTitle": "Senior Application Developer",
    "JobPeriod": "04/2012 - 06/2014"
  },
  {
    "CompanyName6": "iNautix Technologies, US / India",
    "JobTitle": "Application Developer",
    "JobPeriod": "07/2009 - 03/2012"
  }
}


In [5]:
from datetime import datetime
from dateutil import relativedelta
from pydantic import Field

def caculateTimePeriod(fromDateString: str, toDateString: str, dateFormat: str = '%m/%Y'):
    # calulate time period between two dates in the given date format like '%m/%Y'
    fromDate = datetime.strptime(fromDateString, dateFormat)
    toDate = datetime.strptime(toDateString, dateFormat)
    delta = relativedelta.relativedelta(toDate, fromDate)
    duration = f"{delta.years} Years and {delta.months} months"
    return duration


In [12]:
caculateTimePeriod("07/2009", "03/2012", "%m/%Y")

'2 Years and 8 months'

In [6]:
from llama_index.core.tools import FunctionTool

timePeriodFunctionTool = FunctionTool.from_defaults(caculateTimePeriod, name= "caculateTimePeriod")

In [7]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

documents = SimpleDirectoryReader("Data").load_data()
index = VectorStoreIndex.from_documents(documents, show_progress = True)

queryEngineTool = QueryEngineTool.from_defaults(
    index.as_query_engine(), name="ProfileQueryEngine", description="reads profile information from bio data, resume, references so on"
)

In [22]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools([queryEngineTool, timePeriodFunctionTool], verbose=True)

responseFromTool = agent.chat("Where Ramya worked in 2020?\n"
                              "Return the response in Json format with Company name, role, job period information")

print(responseFromTool)

Added user message to memory: where Ramya worked in 2020?
Return the response in Json format with Company name, role, job period information
=== Calling Function ===
Calling function: caculateTimePeriod with args: {"fromDateString":"01/01/2020","toDateString":"12/31/2020"}
Got output: Error: time data '01/01/2020' does not match format '%m/%Y'

=== Calling Function ===
Calling function: caculateTimePeriod with args: {"fromDateString": "01/01/2020", "toDateString": "12/31/2020", "dateFormat": "%m/%d/%Y"}
Got output: 0 Years and 11 months

=== Calling Function ===
Calling function: ProfileQueryEngine with args: {"input": "Ramya work history in 2020"}
Got output: Ramya worked as a Senior Application Developer at Virtusa (CIBC - Capital Market, Canada) from February 2020 to June 2021.

{
  "Company": "Virtusa (CIBC - Capital Market, Canada)",
  "Role": "Senior Application Developer",
  "JobPeriod": "February 2020 to June 2021"
}


In [23]:
responseFromTool1 = agent.chat("Calcluate time period in this job")

print(responseFromTool1)

Added user message to memory: Calcluate time period in this job
=== Calling Function ===
Calling function: caculateTimePeriod with args: {"fromDateString":"02/2020","toDateString":"06/2021","dateFormat":"%m/%Y"}
Got output: 1 Years and 4 months

Ramya worked as a Senior Application Developer at Virtusa (CIBC - Capital Market, Canada) for 1 year and 4 months from February 2020 to June 2021.
